# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key



In [2]:
# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/city.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_df = "../WeatherPy/cities.csv"
weather_df = pd.read_csv(weather_df)
weather_df.head(10)


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
5,5,atuona,78,PF,1558378755,76,-9.80,-139.03,81.96,16.49
6,6,flinders,0,AU,1558378755,62,-34.58,150.85,60.01,11.41
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
8,8,tiksi,95,RU,1558378756,88,71.64,128.87,14.10,15.70
9,9,pushkinskiye gory,6,RU,1558378756,70,57.02,28.91,65.04,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [5]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=300,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
new_city_df = weather_df.loc[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0) & \
                                   (weather_df["Max Temp"] >= 70) & (weather_df["Max Temp"] <= 80)].dropna()

new_city_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
Humidity = weather_df["Humidity"]

# Create a poverty Heatmap layer
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

fig.add_layer(heat_layer)

plt.savefig("vacationheatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

In [8]:
# --- store into variable named hotel_df ---

hotel_df = new_city_df[["City", "Country", "Lat", "Lng"]].copy()

# --- Add a "Hotel Name" column to the DataFrame with null values ---
hotel_df["Hotel Name"] = np.nan
hotel_df

,City,Country,Lat,Lng,Hotel Name
70,ponta do sol,BR,-20.63,-46.00,NaN
88,cidreira,BR,-30.17,-50.22,NaN
138,morondava,MG,-20.30,44.28,NaN
139,vaini,IN,15.34,74.49,NaN
176,nador,MA,35.17,-2.93,NaN
297,mogok,MM,22.92,96.51,NaN
444,birjand,IR,32.86,59.22,NaN


In [9]:
# --- set up parameters for querying Google Places API ---
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# --- set up a parameters dictionary ---
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# --- define base url ---
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# --- use a for loop to go through each city in the dataframe and make an API call ---

for index, row in hotel_df.iterrows():
    
    # --- get the coordinates and store in params dictionary ---
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # --- call the API and get hotel details for each city ---
    hotel_data = requests.get(base_url, params).json()
    
    # --- use exception handling to save the first hotel name for each city ---
    
    try:
        hotel_name = hotel_data["results"][0]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        print(f"City '{row['City']}' found! The nearest hotel is {hotel_name}")
        print("-------------------------------------------------------------------------")
              
    except:
        print(f"Could not find information for city '{row['City']}'... skipping.")    
        print("-------------------------------------------------------------------------")
        

City 'ponta do sol' found! The nearest hotel is Escarpas Resort
-------------------------------------------------------------------------
City 'cidreira' found! The nearest hotel is Hotel Castelo
-------------------------------------------------------------------------
City 'morondava' found! The nearest hotel is Palissandre Cote Ouest resort & SPA
-------------------------------------------------------------------------
City 'vaini' found! The nearest hotel is Dandeli Dreams Jungle Resort
-------------------------------------------------------------------------
City 'nador' found! The nearest hotel is Hôtel Mercure Rif Nador
-------------------------------------------------------------------------
City 'mogok' found! The nearest hotel is Mount Mogok Hotel
-------------------------------------------------------------------------
City 'birjand' found! The nearest hotel is مهمان پذیر نوید
-------------------------------------------------------------------------


In [10]:
# --- remove null rows from dataframe ---
hotel_df.dropna(how="any",inplace=True)
hotel_df.reset_index(drop=True, inplace=True)

# --- display final dataframe ---
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,ponta do sol,BR,-20.63,-46.00,Escarpas Resort
1,cidreira,BR,-30.17,-50.22,Hotel Castelo
2,morondava,MG,-20.30,44.28,Palissandre Cote Ouest resort & SPA
3,vaini,IN,15.34,74.49,Dandeli Dreams Jungle Resort
4,nador,MA,35.17,-2.93,Hôtel Mercure Rif Nador
5,mogok,MM,22.92,96.51,Mount Mogok Hotel
6,birjand,IR,32.86,59.22,مهمان پذیر نوید


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer on top of heat map ---
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))